In [10]:
import os
import openpyxl

def isValidScorePresent(sheet,r,c):
    if sheet.cell(row = r + 1, column=c).value != 'NT' \
            and sheet.cell(row = r + 1, column=c).value != 'I' \
            and sheet.cell(row = r + 1, column=c).value != 'XX' \
            and sheet.cell(row = r + 1, column=c).value != None:
        return True
    else:
        return False

def get_sheetcell_value(sheet,u,i):
    return sheet.cell(row = u+1,column = i).value

print("Collaborative Filtering User-Based Algorithm for Grade Prediction")
wb= openpyxl.load_workbook('student.xlsx')
# print( type(wb))

sheetname = wb.get_sheet_names()
print("Student's score matrix(roll no in first column)\n")
sheet=wb.get_sheet_by_name(sheetname[0])

for i in range(2,sheet.max_row+1,1):
    for j in range(1,sheet.max_column+1,1):
        if sheet.cell(row=i,column=j).value != None:
            if j == 1:
                if int(sheet.cell(row=i,column=j).value/10)==0:
                    print(sheet.cell(row=i, column=j).value, end="  | ")
                else:
                    print(sheet.cell(row=i,column=j).value,end=" |  ")
            else:
                print(sheet.cell(row=i,column=j).value,end="  ")
            if sheet.cell(row=i, column=j).value != 'I' and sheet.cell(row=i, column=j).value != 'XX' and sheet.cell(row=i, column=j).value != 'NT':
                if int(sheet.cell(row=i,column=j).value/10) == 0:
                    print(end=" ")
            if sheet.cell(row=i,column=j).value=='I':
                print(end=" ")
        else:
            print("-",end="   ")
    print()
studentID = int(input("\nEnter studentID : "))

coursesList = []
print("\nAvailable courses for you are as below")
for i in range (2,sheet.max_column+1):
    if (i-2)%9 == 0:
        print()
    print(sheet.cell(row=1,column=i).value,end=", ")
    coursesList.append(sheet.cell(row=1,column=i).value)
print("\n")
while True:
    c = input("\nEnter course: ")

    try:
        course = coursesList.index(c)+1
        break
    except ValueError:
        print("Please select course from given list ")
###---------user based-------
#similarity_between_students b/w u,v
# compute avg grade of every student
def compute_average_score(sheet,student):    # calculate avg grades for student
    sum = 0
    cnt = 0
    for i in range (2,sheet.max_column):
        if isValidScorePresent(sheet,student,i):
            sum = sum + int(get_sheetcell_value(sheet,student,i))
            cnt = cnt + 1
    if cnt != 0:
        return round(float(sum/cnt),3)
    else:
        return 0

#compute similarity score between students 
def similarity_between_students(u, v, sheet,avgU):
    sum1 = 0
    sum2 = 0
    sum3 = 0
    avgV = compute_average_score(sheet, v)
    # print(" avg ",avgU,avgV,v)
    for i in range(2,sheet.max_column+1):
        if isValidScorePresent(sheet,u,i) and isValidScorePresent(sheet,v,i):
            sum1 = sum1 + (get_sheetcell_value(sheet,u,i) - avgU) * (get_sheetcell_value(sheet,v,i) - avgV)
            sum2 = sum2 + (get_sheetcell_value(sheet,u,i) - avgU) * (get_sheetcell_value(sheet,u,i) - avgU)
            sum3 = sum3 + (get_sheetcell_value(sheet,v,i) - avgV) * (get_sheetcell_value(sheet,v,i) - avgV)
    if sum2!=0 and sum3!=0:
        return round(float(sum1/((sum2*sum3)**(1/2))),3)
    else:
        # print("hi ",v)
        return 0


similar_user_val = []
simi_user = []

maxSim=0
most_similar_user=""
highest_similarityBetweenStudents = []
# print(sheet.max_column,sheet.max_row)
avgU = compute_average_score(sheet, studentID)
print("\nBelow is the most similar student based on users based collaborative filtering:")
for i in range(1,sheet.max_row):
    # print(i,sheet.cell(row=i+1,column=1))
    if i!=studentID:
        sim = similarity_between_students(studentID,i,sheet,avgU)
        similar_user_val.append([studentID,i,sim])
        if sim > 0:
            if maxSim<sim:
              maxSim = sim
              most_similar_user =sheet.cell(row=i+1,column=1).value
            studentName1 =sheet.cell(row=studentID+1,column=1).value
            studentName2 =sheet.cell(row=i+1,column=1).value
            simi_user.append([studentID,i,sim])
            highest_similarityBetweenStudents.append([studentName1,studentName2,sim])

print(most_similar_user)         

#predicting grade
sum1 = 0
sum2 = 0
#print(similar_user_val)
#print(highest_similarityBetweenStudents)
for i in range(0,len(simi_user)):
    if isValidScorePresent(sheet,int(simi_user[i][1]),course):
        avg = compute_average_score(sheet,simi_user[i][1])
        sum1 = sum1 + (simi_user[i][2] * (get_sheetcell_value(sheet,simi_user[i][1],course) - avg))
        sum2 = sum2 + simi_user[i][2]
if sum2 != 0:
    print("\nPredicted Grade for studentID "+str(studentID)+" in subject "+str(c)+" : "+str(round(float(sum1/sum2),3)+avgU))

Collaborative Filtering User-Based Algorithm for Grade Prediction
Student's score matrix(roll no in first column)

1  |  4   8   5   5   6   9   0   6   5   3   3   6   5   3   3   3   4   0   0   0   0   4   5   0   4   4   
2  |  0   5   4   4   0   5   4   6   6   5   5   6   6   3   6   5   4   0   0   6   6   4   5   0   5   3   
3  |  0   7   5   5   5   6   0   6   0   4   4   7   5   4   3   4   4   4   3   6   6   4   5   0   4   5   
4  |  10  9   8   8   7   9   9   9   10  9   9   8   8   I   I   4   I   I   I   -   -   -   -   -   -   -   
5  |  4   7   7   7   6   7   3   6   4   4   4   5   5   4   6   4   4   4   3   6   6   5   6   4   3   6   
6  |  4   4   7   7   3   9   5   7   6   6   6   8   5   5   7   0   7   5   6   7   7   3   5   7   6   6   
7  |  3   5   4   4   5   9   4   5   5   4   4   7   7   6   7   7   4   5   3   6   6   4   5   4   7   7   
8  |  5   5   7   7   6   6   3   7   6   5   5   6   5   5   7   6   6   4   5   6   6   4   6   0   5   6 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated function get_sheet_names (Use wb.sheetnames).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).


 |  4   8   5   5   6   8   6   7   9   6   6   6   6   5   8   7   7   4   5   6   6   4   8   4   6   5   
13 |  5   9   7   7   7   6   4   6   6   4   4   7   4   5   7   7   6   6   5   8   8   5   5   4   7   5   
14 |  0   7   5   5   0   5   6   9   8   5   5   6   7   6   7   6   8   5   5   6   6   7   8   5   6   6   
15 |  5   5   5   5   5   5   5   8   8   5   5   5   5   5   5   8   7   6   6   7   7   4   5   5   6   8   
16 |  5   7   6   6   6   9   5   7   8   6   6   8   6   6   6   5   9   5   6   6   6   5   7   4   6   9   
17 |  7   4   6   6   5   9   4   7   8   5   5   6   7   10  4   6   7   5   6   7   7   5   6   7   7   7   
18 |  4   7   5   5   3   7   4   7   6   4   4   6   5   6   5   5   9   5   7   6   6   4   6   0   3   6   
19 |  6   4   4   4   6   8   6   6   8   7   7   5   7   5   7   7   7   6   6   6   6   6   6   5   5   7   
20 |  4   7   5   5   3   10  3   6   6   7   7   4   9   4   7   8   6   7   6   6   6   7   7   7   6   5   
21 